In [32]:
%matplotlib inline
import numpy as np
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from random import sample
from collections import Counter
import string
from sklearn.dummy import DummyClassifier
from numpy.random import random_integers
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
from scipy import sparse
import time

In [3]:
def readCsv(fileName):
    fullFileName = r'C:\Users\Owner\McGill\4thYear\COMP551\final-project\Datasets\dbpedia_csv' + fileName
    df = pd.read_csv(fullFileName, encoding='utf-8', header = None, names = ['y', 'title','desc'],
                 sep=',')
    
    return df

In [4]:
dfTrain = readCsv(r'\train.csv')
y_train = dfTrain['y']
x_train = dfTrain.drop('y', axis=1)
x_train = x_train[['title','desc']].apply(lambda x: ''.join(x), axis=1)

dfTest = readCsv(r'\test.csv')
y_test = dfTest['y']
x_test = dfTest.drop('y', axis=1)
x_test = x_test[['title','desc']].apply(lambda x: ''.join(x), axis=1)

In [19]:
dicLen = 5000
def createDictionary():
    allWords = list()
    cnt = Counter()
    
# Replacing !"#$%&()*+,-./:;<=>?@[\]^_`{|}~ with ' ' * 31 (31 spaces, needs to be same length)
# and replacing ' with ' ' (apostrophe with space)
    translator = str.maketrans(string.punctuation.replace('\'', ''), 31*' ', '\'')
    
    for i in range(0, len(x_train)):
        allWords.extend(x_train.iloc[i].translate(translator).lower().split(" "))
    
    for word in allWords:
        cnt[word] +=1
    
    dictionaryWords = list(zip(*cnt.most_common(dicLen + 1)[1:]))[0]
    
    dictionary = {}
    for index, key in enumerate(dictionaryWords):
        dictionary[key] = index
#     print(dictionary)
    newArray = np.asarray(cnt.most_common(dicLen + 1)[1:])
    withIndexVocabArray = np.insert(newArray, 1, range(0,dicLen),1)
    
#     if trainingSet=='IMDB':
#         toCsvDf('\IMDB-vocab.txt', pd.DataFrame(withIndexVocabArray))
#     elif trainingSet=='yelp':
#         toCsvDf('\yelp-vocab.txt', pd.DataFrame(withIndexVocabArray))
        
    return dictionary
dictionary = createDictionary()

In [10]:
def convertWordsToVector(trainingRow, dictionary, BOWType):
    translator = str.maketrans(string.punctuation.replace('\'', ''), 31*' ', '\'')
    returnRow = trainingRow.translate(translator).lower().split(" ")
    vector = np.zeros(len(dictionary), dtype = np.int8)
    
    for word in returnRow:
        if word in dictionary:
            if BOWType == 'BagOfWords':
                vector[dictionary[word]] = np.int8(1)
            elif BOWType == 'Frequency':
                vector[dictionary[word]] += 1
    
    if BOWType == 'Frequency':
#         To accomodate for the fact that one of the rows has one word, d-gust-ing, and the lenght of the vector is zero.
#         Hence, the vector returns [Nan Nan Nan ... Nan Nan].  Now it returns [0 0 0 ... 0 0 0]
        vectorLength =  np.sum(vector)
        if vectorLength>0:
            vector = np.divide(vector, vectorLength)
            
    return vector

In [11]:
def createBagOfWordsMatrix(df, dictionary, BOWType):
    translator = str.maketrans(string.punctuation.replace('\'', ''), 31*' ', '\'')
    
    if BOWType == 'BagOfWords':
        distancesArray = np.zeros((len(df), len(dictionary)), dtype = np.int8)
    elif BOWType == 'Frequency':
        distancesArray = np.zeros((len(df), len(dictionary)))
        
    for i in range(0, len(df)):
        vector = convertWordsToVector(df.iloc[i], dictionary, BOWType)
        distancesArray[i] = vector
        
    return distancesArray

# print(createBagOfWordsMatrix(readTxt('\yelp-train.txt'), createDictionary('yelp'), 'Frequency'))

In [14]:
def testScores(predicted, actual):
    f1Score = f1_score(actual, predicted, average='macro')
    accuracy = accuracy_score(actual, predicted)
    confusionMatrix = confusion_matrix(actual, predicted)
    print('F1 Score:', f1Score)
    print('Accuracy:', accuracy)
    print('Confusion Matrix:\n', confusionMatrix)
    return f1Score

In [20]:
def bernoulliNB(dataset, BOWType):
    print(dataset)
#     dictionary = createDictionary()
    x_train_matrix = sparse.csr_matrix(createBagOfWordsMatrix(x_train, dictionary, BOWType))
    x_test_matrix = sparse.csr_matrix(createBagOfWordsMatrix(x_test, dictionary, BOWType))
    
    clf = BernoulliNB()
    print(clf.get_params())
    clf.fit(x_train_matrix, y_train)
    
    print('Training F1-Measure')
    predictionsArray = clf.predict(x_train_matrix)
    testScores(predictionsArray, y_train)
    
#     print('Validation F1-Measure')
#     predictionsArray = clf.predict(createBagOfWordsMatrix(validDF, dictionary, BOWType))
#     testScores(predictionsArray, validDF[1])
    
    print('Test F1-Measure')
    predictionsArray = clf.predict(x_test_matrix)
    testScores(predictionsArray, y_test)
    
bernoulliNB('yelp', 'BagOfWords')

yelp
{'alpha': 1.0, 'binarize': 0.0, 'class_prior': None, 'fit_prior': True}
Training F1-Measure
F1 Score: 0.941151126164
Accuracy: 0.941294642857
Confusion Matrix:
 [[35628   538   412    19    56   560   451   209     3    53    26   775
    212  1058]
 [  336 38828    55    20    56    24   402   152     2    18     1     0
      5   101]
 [  148    34 36575   201   798    14    44    32     1    55     2   808
    292   996]
 [   18     3   382 39453    99     8     3     7     0     8     2     1
      3    13]
 [  102    80   676   128 38749    50    47    49     0    49     0     0
     10    60]
 [ 1131    21    14    21     4 38402   142   104     0    50     2     8
     39    62]
 [ 1099  1895   122    19   101   164 34936  1426    20    60     5     4
     14   135]
 [   55    30     4     3     4    76   483 39148    11   104    36     2
      1    43]
 [   32   455     5     1    16     4   492  1565 37403     4     1     0
      1    21]
 [   25     7    96   479    37  

In [33]:
def linearSVC(dataset, BOWType):
    print(dataset)
#     dictionary = createDictionary()
    x_train_matrix = sparse.csr_matrix(createBagOfWordsMatrix(x_train, dictionary, BOWType))
    x_test_matrix = sparse.csr_matrix(createBagOfWordsMatrix(x_test, dictionary, BOWType))
    
    t0 = time.time()
    clf = LinearSVC()
    print(clf.get_params())
    clf.fit(x_train_matrix, y_train)
    
    print('Training F1-Measure')
    predictionsArray = clf.predict(x_train_matrix)
    testScores(predictionsArray, y_train)
    
#     print('Validation F1-Measure')
#     predictionsArray = clf.predict(createBagOfWordsMatrix(validDF, dictionary, BOWType))
#     testScores(predictionsArray, validDF[1])
    
    print('Test F1-Measure')
    predictionsArray = clf.predict(x_test_matrix)
    testScores(predictionsArray, y_test)
    
    t1 = time.time()
    print('Total time: %f' %(t1-t0))
    
linearSVC('yelp', 'BagOfWords')

yelp
{'C': 1.0, 'class_weight': None, 'dual': True, 'fit_intercept': True, 'intercept_scaling': 1, 'loss': 'squared_hinge', 'max_iter': 1000, 'multi_class': 'ovr', 'penalty': 'l2', 'random_state': None, 'tol': 0.0001, 'verbose': 0}
Training F1-Measure
F1 Score: 0.988685591834
Accuracy: 0.988691071429
Confusion Matrix:
 [[38717   253    95     4    34   124   391    20     1    16    10    30
     30   275]
 [  177 39553     8     0    17     1   202    12     4     4     1     0
      1    20]
 [  144    21 39089    40   330     3    38     2     1    11     3    60
     45   213]
 [    3     1    44 39928    20     1     0     1     0     1     0     0
      0     1]
 [   49    21   332    15 39549     2    16     2     0     4     1     0
      2     7]
 [   87     4     3     0     1 39872    26     4     0     2     1     0
      0     0]
 [  376   291    28     2    22    26 39128    84    11     4     4     0
      2    22]
 [    5     5     0     0     0     2    43 39917    17 

In [30]:
t0 = 10
t1 = 20
print('total time %f' %(t1-t0))

total time 10.000000


In [ ]:
def iterateBernoulliNB(dataset, BOWType):
    print(dataset)
    print(BOWType)
    if dataset == 'yelp':
        dictionary = createDictionary('yelp')
        trainingDF = readTxt('\yelp-train.txt')
        validDF = readTxt('\yelp-valid.txt')
        testDF = readTxt('\yelp-test.txt')
    elif dataset == 'IMDB':
        dictionary = createDictionary('IMDB')
        trainingDF = readTxt('\IMDB-train.txt')
        validDF = readTxt('\IMDB-valid.txt')
        testDF = readTxt('\IMDB-test.txt')
    
    trainingDataMatrix = createBagOfWordsMatrix(trainingDF, dictionary, BOWType)
    validDataMatrix = createBagOfWordsMatrix(validDF, dictionary, BOWType)
    testDataMatrix = createBagOfWordsMatrix(testDF, dictionary, BOWType)
    targetVectorTraining = trainingDF[1]
    targetVectorValid = validDF[1]
    targetVectorTest = testDF[1]
    
    alphaLinspace = np.linspace(0.000001, 0.5, 40)
    f1Scores = list()
    
#     tuning parameters with validation set
    for alpha in alphaLinspace:
        clf = BernoulliNB(alpha = alpha)
        clf.fit(trainingDataMatrix, targetVectorTraining)
        predictionsArray = clf.predict(validDataMatrix)
        f1Score = testScores(predictionsArray, targetVectorValid)
#         print(f1Score)
        f1Scores.append(f1Score)
    
    plt.figure(figsize=(20,8), dpi=80)
    plt.plot(alphaLinspace, f1Scores)
    plt.axvline(x=alphaLinspace[np.argmax(f1Scores)],linestyle='dashed')
    plt.xlabel('Alpha')
    plt.ylabel('F1-Measure')
    plt.title('F1-Measure Validation')
    plt.show()

    bestF1Alpha = alphaLinspace[np.argmax(f1Scores)]
    print('Best Alpha for F1-Measure ', bestF1Alpha)
    print('MAX F1-Measure ', max(f1Scores))
    
#     final Test F1-Measure
    
    clf = BernoulliNB(alpha = bestF1Alpha)
    clf.fit(trainingDataMatrix, targetVectorTraining)
    testPredictionsArray = clf.predict(testDataMatrix)
    testF1Score = testScores(testPredictionsArray, targetVectorTest)
    print('F1 Score for Testing at Alpha = {0} is {1}'.format(bestF1Alpha, testF1Score))
    
    return alphaLinspace[np.argmax(f1Scores)]

    
        
# iterateBernoulliNB('IMDB', 'Frequency')
# iterateBernoulliNB('IMDB', 'BagOfWords')
# iterateBernoulliNB('IMDB', 'Frequency')

In [12]:
def randomClassifier(dataset, BOWType):
    print(dataset, 'Random Classifier')
    
    clf = DummyClassifier(strategy='uniform')
    print(clf.get_params())
    clf.fit(createBagOfWordsMatrix(x_train, dictionary, BOWType), y_train)
    
    print('Training F1-Measure')
    predictionsArray = clf.predict(createBagOfWordsMatrix(x_train, dictionary, BOWType))
    testScores(predictionsArray, y_train)
    
#     print('Validation F1-Measure')
#     predictionsArray = clf.predict(createBagOfWordsMatrix(validDF, dictionary, BOWType))
#     testScores(predictionsArray, validDF[1])
    
    print('Test F1-Measure')
    predictionsArray = clf.predict(createBagOfWordsMatrix(x_test, dictionary, BOWType))
    testScores(predictionsArray, y_test)
    
randomClassifier('yelp', 'BagOfWords')

yelp Random Classifier
{'constant': None, 'random_state': None, 'strategy': 'uniform'}
Training F1-Measure
F1 Score: 0.0717236247768
Confusion Matrix:
 [[2856 2963 2860 2904 2894 2873 2831 2849 2771 2732 2852 2838 2877 2900]
 [2798 2793 2845 2962 2894 2863 2971 2809 2838 2838 2866 2870 2810 2843]
 [2900 2944 2908 2797 2862 2780 2866 2816 2916 2823 2851 2776 2858 2903]
 [2886 2842 2859 2763 2804 2828 2890 2828 2907 2885 2919 2880 2885 2824]
 [2793 2919 2851 2891 2907 2854 2839 2812 2902 2800 2798 2895 2890 2849]
 [2836 2897 2834 2873 2837 2851 2953 2833 2860 2829 2851 2884 2829 2833]
 [2893 2942 2905 2790 2776 2876 2851 2834 2851 2784 2874 2874 2882 2868]
 [2827 2778 2893 2866 2847 2855 2875 2857 2891 2894 2847 2797 2906 2867]
 [2828 2812 2827 2828 2848 2866 2887 2884 2939 2779 2845 2933 2900 2824]
 [2850 2811 2917 2806 2910 2909 2860 2927 2863 2796 2882 2807 2825 2837]
 [2899 2848 2905 2823 2957 2888 2857 2845 2844 2761 2899 2838 2825 2811]
 [2828 2822 2838 2865 2842 2743 2912 2956 290